In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os
import geopandas as gpd # this is the main geopandas 

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

In [2]:
background = "#ffffff"

month = "10"

In [3]:
def load_map():
    
    map_projection = "epsg:2163"
    
    county_shape = ".\\shapefiles\\lake\\ne_10m_lakes.shx"

    lake_map = gpd.read_file(county_shape)

    lake_map = lake_map.to_crs({'init': map_projection})
    
##########################################################################
    
    county_shape = ".\\shapefiles\\land\\ne_50m_land.shx"

    land_map = gpd.read_file(county_shape)

    land_map = land_map.to_crs({'init': map_projection})

    land_map = land_map.iloc[0:1200]
    
##########################################################################
    
    county_shape = ".\\shapefiles\\county\\tl_2017_us_county.shx"

    us_map = gpd.read_file(county_shape)

    us_map = us_map.to_crs({'init': map_projection})
    
    us_map["geometry"] = us_map["geometry"].simplify(400)
    
    us_map["area_fips"] = (us_map.STATEFP.astype(str) + us_map.COUNTYFP.astype(str)).astype(int)

    us_map.set_index("STATEFP", inplace = True)

    drop_list = ["02","15","72"]

    us_map.drop(drop_list, inplace = True)
    
    us_map.reset_index(inplace = True)
    
##########################################################################
    
    state_shape = ".\\shapefiles\\state\\tl_2017_us_state.shx"

    state_map = gpd.read_file(state_shape)

    state_map = state_map.to_crs({'init': map_projection})

    state_map["geometry"] = state_map["geometry"].simplify(400)

    state_map.set_index("STATEFP", inplace = True)

    drop_list = ["02","15","72","78","69","66","60",]

    state_map.drop(drop_list, inplace = True)
    
    state_map.reset_index(inplace = True)
    
    state_fp_dict = dict(zip(state_map.STATEFP, state_map.STUSPS))
    
    us_map["STSPS"] = us_map["STATEFP"].map(state_fp_dict)
    
    us_map["NAME"] = us_map["NAME"] + " County, " + us_map["STSPS"]
    
##########################################################################
    
    us_map = gpd.overlay(us_map, land_map,  how='intersection')
    
    great_lakes = ["Lake Superior", "Lake Michigan", "Lake Erie","Lake Superior""Lake Huron"]

    us_map = gpd.overlay(us_map, lake_map[lake_map.name.isin(great_lakes)],  how='difference')
    
    state_map = gpd.overlay(state_map, land_map,  how='intersection')

    state_map = gpd.overlay(state_map, lake_map[lake_map.name.isin(great_lakes)],  how='difference')
    
    return us_map, state_map

In [4]:
def make_county_source(month):
    file = ".\\data\\" + "phase_one_county.parquet"
    
    df = pq.read_table(file).to_pandas()

    df.reset_index(inplace = True)

    df["time"] = pd.to_datetime(df.time)

    df.set_index(["area_fips", "time"],inplace = True)
    
    grp = df.groupby(["area_fips"])
    
    foo = grp.apply(county_trade_ytd_gain, month)
    
    foo = foo.droplevel(1)
    
    foo.reset_index(inplace = True)
    
    foo["fips_code"] = foo["fips"].astype(int)
    
    foo = foo[["fips_code", "china_exp_pc","china_pho_pc", "2017_population", "phaseone_gain", "2017_exports_ytd", "ytd_exports"]]

    foo["pop_label"] = foo["2017_population"].map('{:,.0f}'.format)
    
    foo["ytd_label"] = foo["ytd_exports"].map('{:,.1f}'.format)
    
    foo["2017_ytd_label"] = foo["2017_exports_ytd"].map('{:,.1f}'.format)

    foo["phaseone_gain_label"] = foo["phaseone_gain"].map('{:,.1f}'.format)
    
    return foo

In [5]:
def county_trade_ytd_gain(df, month):
    
    year2020 = "2020-" + month + "-01"
    year2017 = "2017-" + month + "-01"
    
    idx = pd.IndexSlice
    
    df["phaseone_gain"] = np.nan
    df["ytd_exports"] = np.nan
    df["2017_exports_ytd"] = np.nan
    
    df["phaseone_gain"].loc[idx[:,year2020]] = 100* ((df.china_pho_pc.loc[idx[:,"2020-01-01":year2020]].sum() 
                                        / df.china_pho_pc.loc[idx[:,:year2017]].sum()) - 1)
    
    df["ytd_exports"].loc[idx[:,year2020]] = df.china_pho_pc.loc[idx[:,"2020-01-01":year2020]].sum() 
    
    df["2017_exports_ytd"].loc[idx[:,year2020]] = df.china_pho_pc.loc[idx[:,:year2017]].sum()
    
    return df.loc[idx[:,year2020],:]

In [6]:
def make_map(df, data_series):
    
    width = 900
    height = 600
    #df = make_county_source(time)
        
    df.phaseone_gain.fillna(0, inplace = True)
    
    var = ["phaseone_gain","fips_code","pop_label","phaseone_gain_label", "2017_ytd_label", "ytd_label",
          "2017_exports_ytd", "ytd_exports"]
    
    us_map = us_map_fixed.merge(df[var], left_on='area_fips',
                      right_on = "fips_code", how = "inner", indicator = True)
    ################################################################################
        
    state_geosource = GeoJSONDataSource(geojson = state_map.to_json())

    geosource = GeoJSONDataSource(geojson = us_map.to_json())
    
    palette = brewer['RdBu'][10]
    
    if data_series == "phaseone_gain":
        color_mapper = LinearColorMapper(palette = palette, low = -75, high = 75)
        title = "2020 Year to Date County-Level Exports to China Relative to 2017 YTD,  % Change"
    
    if data_series == "2017_exports_ytd":
        color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1200)
        title = "2017 Year to Date County-Level Exports per Worker"
        
    if data_series == "ytd_exports":
        color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1200)
        title = "2020 Year to Date County-Level Exports per Worker"

    color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0))
    #color_bar.sizing_mode= "scale_both"
    
    
# Create figure object.
    p = figure( 
           plot_height = height,
           plot_width = width, 
           toolbar_location = 'below',
           tools = "box_zoom, reset",
        title = title)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    states = p.patches('xs','ys', source = geosource,
                   fill_color = {"field" : data_series,
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

    state_line = p.multi_line('xs','ys', source = state_geosource,
                   line_color = "black", 
                   line_width = 0.5)

    p.axis.visible = False
    p.background_fill_color = "grey"
    p.background_fill_alpha = 0.025
    
    TOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.85;">
        <div style = "text-align:center;">
            <span style="font-size: 13px; font-weight:;"><b>@NAME</b> (pop. @pop_label) </span>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 13px; font-weight:">Exports ($) per worker, YTD. </span>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 13px; font-weight:">2017: <b>@2017_ytd_label</b> &nbsp;&nbsp;&nbsp;&nbsp; 2020: <b>@ytd_label</b> </span>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 13px; font-weight: ">% difference: <b>@phaseone_gain_label</b></span>
        </div>
    </div>
    """
    p.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))
    
    p.title.text_font_size = '13pt'
    p.toolbar.autohide = True
    p.add_layout(color_bar, "left")
    p.border_fill_color = background
    color_bar.background_fill_color = background
    p.background_fill_color = background
    p.background_fill_alpha = 1.0
    p.toolbar.active_drag = None
    
    p.outline_line_color = None
    
    divsp = Div(text = """ """, width=100, background = background)
    divsp.sizing_mode= "scale_width"
    
    div0 = Div(text="""<p style="text-align:full;"> 
    Each county is color coded according to the year to date (YTD) percent increase 
    in exports per worker to China, relative to 2017 (same YTD). Only phase one (annext 6-1) coverd products are included.
    The hover tool reports the county name, levels, and the percent change.</p> """
    , width=275, background = background)
    div0.sizing_mode= "scale_width"
    
    div1 = Div(text="""<p style="text-align:full;"> 
    County-level exports are computed by apportioning industry-level national exports
    to a county based on that county's employment as a share of national employment in
    the industry. For example, if one county has all the nation's soybean production 
    employment, then all of soybean exports are assigned to that county. The code 
    repository and Waugh (2019) provide more details.</p> """, width=275, background = background )
    div1.sizing_mode= "scale_width"
    
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    
    p = row(p)
    
    #show(p)
    return p

In [7]:
us_map_fixed, state_map = load_map()

In [8]:
df = make_county_source(month)

In [9]:
df.head()

,fips_code,china_exp_pc,china_pho_pc,2017_population,phaseone_gain,2017_exports_ytd,ytd_exports,pop_label,ytd_label,2017_ytd_label,phaseone_gain_label
0,10001,111.762101,107.213139,173145.0,32.641917,387.657803,514.196742,"173,145",514.2,387.7,32.6
1,10003,44.692063,30.023913,555036.0,23.509752,219.044056,270.540769,"555,036",270.5,219.0,23.5
2,10005,144.005283,129.805111,215551.0,29.674367,558.627281,724.396389,"215,551",724.4,558.6,29.7
3,1001,164.208348,89.933922,55036.0,-44.154282,888.281833,496.067365,"55,036",496.1,888.3,-44.2
4,1003,129.368765,112.596974,203360.0,-3.862852,470.945270,452.753351,"203,360",452.8,470.9,-3.9


In [10]:
df.head()

,fips_code,china_exp_pc,china_pho_pc,2017_population,phaseone_gain,2017_exports_ytd,ytd_exports,pop_label,ytd_label,2017_ytd_label,phaseone_gain_label
0,10001,111.762101,107.213139,173145.0,32.641917,387.657803,514.196742,"173,145",514.2,387.7,32.6
1,10003,44.692063,30.023913,555036.0,23.509752,219.044056,270.540769,"555,036",270.5,219.0,23.5
2,10005,144.005283,129.805111,215551.0,29.674367,558.627281,724.396389,"215,551",724.4,558.6,29.7
3,1001,164.208348,89.933922,55036.0,-44.154282,888.281833,496.067365,"55,036",496.1,888.3,-44.2
4,1003,129.368765,112.596974,203360.0,-3.862852,470.945270,452.753351,"203,360",452.8,470.9,-3.9


In [11]:
output_file('.\\docs\\' + "phase_one_map.html")

tab1 = Panel(child= make_map(df, "phaseone_gain"), title="2020 vs 2017 YTD, % Change")

tab2 = Panel(child= make_map(df, "ytd_exports"), title="2020 Exports YTD, in $")

tab3 = Panel(child= make_map(df, "2017_exports_ytd"), title="2017 Exports YTD, in $")

tabs = Tabs(tabs=[tab1, tab2, tab3])

show(tabs)

In [12]:
df.to_csv(".\\data\\phaseone-map.csv")

zipObj = zf.ZipFile('.\\data\\phaseone-trade-map.zip', 'w')
 
# Add multiple files to the zip
zipObj.write('.\\data\\phaseone-map.csv')